<a href="https://colab.research.google.com/github/ArifAygun/CustomerEye/blob/main/Company_Reviews(Master2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### IMPORT LIBRARIES AND DATASET

In [18]:
!pip install pycountry
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 5.5 MB/s eta 0:00:00


In [19]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates
import pycountry

import emoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Analysis_1/

# Read CSV files into DataFrames
df1 = pd.read_csv('Freedom_Debt_Relief2.csv')
df2 = pd.read_csv('Millennium_Trust2.csv')
df3 = pd.read_csv('Advance_America2.csv')

print(df1.info(), df1.head())
print(df2.info(), df2.head())
print(df3.info(), df3.head())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Analysis_1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             2000 non-null   float64
 1   Experience Date  2000 non-null   object 
 2   Review Date      2000 non-null   object 
 3   Reply Date       2000 non-null   object 
 4   Exp to Review    2000 non-null   int64  
 5   Review to Reply  2000 non-null   int64  
 6   Rating           2000 non-null   int64  
 7   Countries        2000 non-null   object 
 8   Reviews          2000 non-null   object 
 9   Replies          2000 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 156.4+ KB
None      Year Experience Date Review Date  Reply Date  Exp to Review  \
0  2021.0      2021-04-30  2021-04-30 

## TEXT PREPROCESSING


Before performing NLP tasks, it's important to preprocess the text data, which typically involves removing stopwords, punctuation, and converting text to lowercase.

In [21]:
def preprocess_text_for_sentiment_analysis(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Handle emojis
    words = [emoji.demojize(word, delimiters=('', ' ')) for word in words]

    # Join tokens back into a string
    cleaned_text = ' '.join(words)

    return cleaned_text

# Apply the preprocessing function to the "Reviews" column in each DataFrame
df1['Cleaned_Reviews'] = df1['Reviews'].apply(preprocess_text_for_sentiment_analysis)
df2['Cleaned_Reviews'] = df2['Reviews'].apply(preprocess_text_for_sentiment_analysis)
df3['Cleaned_Reviews'] = df3['Reviews'].apply(preprocess_text_for_sentiment_analysis)

# Display the cleaned data
print("Cleaned DataFrame 1:")
print(df1[['Reviews', 'Cleaned_Reviews']].head())

print("\nCleaned DataFrame 2:")
print(df2[['Reviews', 'Cleaned_Reviews']].head())

print("\nCleaned DataFrame 3:")
print(df3[['Reviews', 'Cleaned_Reviews']].head())


Cleaned DataFrame 1:
                                             Reviews  \
0  FDR OVERSIGHT I typically would give FDR a 5 s...   
1  Freedom has really helped with my debt… Freedo...   
2  Everything is following through just as… Every...   
3  Started the program owing a little… Started th...   
4  I was surprised to have a settlement so… I was...   

                                     Cleaned_Reviews  
0  fdr oversight typic would give fdr star rate f...  
1  freedom realli help debt freedom realli help d...  
2  everyth follow everyth follow explain would be...  
3  start program owe littl start program owe litt...  
4      surpris settlement surpris settlement quickli  

Cleaned DataFrame 2:
                                             Reviews  \
0                  Website was easy to navigate. nan   
1                           5 star Excellent service   
2  Moving monies Getting all my investments in on...   
3  The service was excellent The service was exce...   
4  User Fr